# <ins>{Project Title}</ins>
#### Group Members: Anthony Rojas, Alexander Parks, Monique Tran & Itzhak Estrella
<img src="andrey-metelev-powergrid.jpg" width="600" height="400" align="center"/>

*(Andrey Matelev, 2021)*


## Introduction
---
The use of Machine Learning and Artificial Intelligence techniques has gained significant importance in recent years, offering a lot of potential in the transition towards renewable and non-renewable energy-based electrical infrastructures. One of the most promising areas for the application of these technologies is the forecasting of energy consumption and generation in energy markets. This project aims to explore a set of comprehensive datasets that contain four years worth of data related to electrical consumption, generation, pricing, and weather conditions in Spain. These sets of datasets are essential for understanding and predicting energy market dynamics and contributing to the transition to sustainable energy sources.

## Dataset Overview:
---
The datasets we are working with here provides valuable information pertaining to the Spanish energy market. The data includes unique attributes/variables such as electrical consumption, generation, pricing, and weather conditions. The following data was gathered from various reputable sources:

- **Electrical Consumption and Generation Data:** This dataset was collected from *European Network of Transmission System Operators for Electricity* (ENTSOE), a public portal for Transmission Service Operator (TSO) data.

- **Settlement Prices:** The settlement prices were obtained from the Spanish TSO, *Red Electric España* (REE).

- **Weather Data:** The weather data for the five largest cities in Spain was sourced from [Open Weather API](https://openweathermap.org/api).

## Objective
---
In this project, we will explore, clean, visualize, and analyze this dataset to gain insights into the energy market in Spain. Our objective is to ...

In [627]:
# Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [628]:
energy_file = 'energy_dataset.csv'
weather_file = 'weather_features.csv'

energy_df = pd.read_csv(energy_file)
weather_df = pd.read_csv(weather_file)

# Exploring the Energy Dataset

In [629]:
energy_df.head()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation marine,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,NaN,863.0,1051.0,1899.0,0.0,7096.0,43.0,73.0,49.0,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,NaN,920.0,1009.0,1658.0,0.0,7096.0,43.0,71.0,50.0,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,NaN,1164.0,973.0,1371.0,0.0,7099.0,43.0,73.0,50.0,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,NaN,1503.0,949.0,779.0,0.0,7098.0,43.0,75.0,50.0,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,NaN,1826.0,953.0,720.0,0.0,7097.0,43.0,74.0,42.0,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04


In [630]:
print('Shape of Energy Dataset {}\n'.format(energy_df.shape))
print(energy_df.info())

''' 
Documentation/Comments:
- This dataframe contains one column ('generation hydro pumped storage aggregated') that contains no value
- The following columns (6/29) except these contain NO missing values: 
    'time', 'forecast solar day ahead', 'forecast wind onshore day ahead', 'total load forecast', 'price day ahead', 'price actual' 
- All energy data is presented as float type
'''

Shape of Energy Dataset (35064, 29)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 29 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   time                                         35064 non-null  object 
 1   generation biomass                           35045 non-null  float64
 2   generation fossil brown coal/lignite         35046 non-null  float64
 3   generation fossil coal-derived gas           35046 non-null  float64
 4   generation fossil gas                        35046 non-null  float64
 5   generation fossil hard coal                  35046 non-null  float64
 6   generation fossil oil                        35045 non-null  float64
 7   generation fossil oil shale                  35046 non-null  float64
 8   generation fossil peat                       35046 non-null  float64
 9   generation geothermal              

" \nDocumentation/Comments:\n- This dataframe contains one column ('generation hydro pumped storage aggregated') that contains no value\n- The following columns (6/29) except these contain NO missing values: \n    'time', 'forecast solar day ahead', 'forecast wind onshore day ahead', 'total load forecast', 'price day ahead', 'price actual' \n- All energy data is presented as float type\n"

# Exploring Weather Data

In [631]:
print('Our Given Five Spanish Cities:', weather_df['city_name'].unique())
weather_df.head()
w_barcelona_df = weather_df[weather_df['city_name'] == 'Barcelona']

Our Given Five Spanish Cities: ['Valencia' 'Madrid' 'Bilbao' ' Barcelona' 'Seville']


#### Units of Measurement for Weather Parameters
| Attribute            | Unit Measurement           |
|----------------------|---------------------------|
| temp                 | °K (Kelvin)              |
| pressure             | hPa (Hectopascal)          |
| humidity             | % (Percentage)            |
| wind_speed           | m/s (Meters per second)   |
| wind_deg             | ° (Degrees)               |
| rain_1h              | mm/h (Millimeters per hour)          |
| rain_3h              | mm (Millimeters)          |
| snow_3h              | mm (Millimeters)          |
| clouds_all           | % (Percentage)            |
* *[Documentation](https://openweathermap.org/weather-data)*


In [632]:
print('Shape of Weather Dataset {}\n'.format(weather_df.shape))
print(weather_df.info())

''' 
Documentation/Comments:
- This dataframe contains no missing (NULL) values
- Data types present in this dataframe: float(6), int(6), sting objects(5)
'''

Shape of Weather Dataset (178396, 17)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178396 entries, 0 to 178395
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   dt_iso               178396 non-null  object 
 1   city_name            178396 non-null  object 
 2   temp                 178396 non-null  float64
 3   temp_min             178396 non-null  float64
 4   temp_max             178396 non-null  float64
 5   pressure             178396 non-null  int64  
 6   humidity             178396 non-null  int64  
 7   wind_speed           178396 non-null  int64  
 8   wind_deg             178396 non-null  int64  
 9   rain_1h              178396 non-null  float64
 10  rain_3h              178396 non-null  float64
 11  snow_3h              178396 non-null  float64
 12  clouds_all           178396 non-null  int64  
 13  weather_id           178396 non-null  int64  
 14  weather_main         178396 n

' \nDocumentation/Comments:\n- This dataframe contains no missing (NULL) values\n- Data types present in this dataframe: float(6), int(6), sting objects(5)\n'

## Merge Datasets

In [633]:
# Lets check the sizes of each subset df within Spanish city:

# Valencia
w_valencia_df = weather_df[weather_df['city_name'] == 'Valencia']
print('Valencias Weather Shape:', w_valencia_df.shape)
# Madrid
w_madrid_df =  weather_df[weather_df['city_name'] == 'Madrid']
print('Madrid Weather Shape:', w_madrid_df.shape)

# Bilbao
w_bilbao_df = weather_df[weather_df['city_name'] == 'Bilbao']
print('Bilbao Weather Shape:', w_bilbao_df.shape)

# Barcelona
w_barcelona_df = weather_df[weather_df['city_name'] == ' Barcelona']
print('Barcelona Weather Shape:', w_barcelona_df.shape)

# Seville
w_seville_df = weather_df[weather_df['city_name'] == 'Seville']
print('Seville Weather Shape:', w_seville_df.shape)

Valencias Weather Shape: (35145, 17)
Madrid Weather Shape: (36267, 17)
Bilbao Weather Shape: (35951, 17)
Barcelona Weather Shape: (35476, 17)
Seville Weather Shape: (35557, 17)


In [634]:
# Lets check if they start and end with the same data + time
print(w_valencia_df['dt_iso'].iloc[0] == w_madrid_df['dt_iso'].iloc[0] == w_bilbao_df['dt_iso'].iloc[0] == w_barcelona_df['dt_iso'].iloc[0] == w_seville_df['dt_iso'].iloc[0])
print(w_valencia_df['dt_iso'].iloc[-1] == w_madrid_df['dt_iso'].iloc[-1] == w_bilbao_df['dt_iso'].iloc[-1] == w_barcelona_df['dt_iso'].iloc[-1] == w_seville_df['dt_iso'].iloc[-1])
# All start at the same date + time AND end at the same date + time

# Now lets use set comparisons
set_valencia = set(w_valencia_df['dt_iso'])
set_madrid = set(w_madrid_df['dt_iso'])
set_bilbao = set(w_bilbao_df['dt_iso'])
set_barcelona = set(w_barcelona_df['dt_iso'])
set_seville = set(w_seville_df['dt_iso'])

sets = [set_valencia, set_madrid, set_bilbao, set_barcelona, set_seville]

# Calculate the intersection of all sets
intersection_dt = set_valencia.intersection(set_madrid,set_bilbao, set_barcelona, set_seville)

# Modify each set to only contain elements in the intersection
for s in sets:
    s.intersection_update(intersection_dt)

# Print the modified sets
for i, s in enumerate(sets, start=1):
    print(f"Set {i}: {len(s)}")

print(len(intersection_dt)) 

True
True
Set 1: 35064
Set 2: 35064
Set 3: 35064
Set 4: 35064
Set 5: 35064
35064


In [635]:
# Valencia
w_valencia_df = w_valencia_df[w_valencia_df['dt_iso'].isin(intersection_dt)]
w_valencia_df = w_valencia_df.drop_duplicates(subset=['dt_iso'])
# Madrid
w_madrid_df = w_madrid_df[w_madrid_df['dt_iso'].isin(intersection_dt)]
w_madrid_df = w_madrid_df.drop_duplicates(subset=['dt_iso'])

# Bilbao
w_bilbao_df = w_bilbao_df[w_bilbao_df['dt_iso'].isin(intersection_dt)]
w_bilbao_df = w_bilbao_df.drop_duplicates(subset=['dt_iso'])

# Barcelona
w_barcelona_df = w_barcelona_df[w_barcelona_df['dt_iso'].isin(intersection_dt)]
w_barcelona_df = w_barcelona_df.drop_duplicates(subset=['dt_iso'])

# Seville
w_seville_df = w_seville_df[w_seville_df['dt_iso'].isin(intersection_dt)]
w_seville_df = w_seville_df.drop_duplicates(subset=['dt_iso'])


print('Valenci\'as Weather Shape:', w_valencia_df.shape)
print('Madrid\'s Weather Shape:', w_madrid_df.shape)
print('Bilbao\'s Weather Shape:', w_bilbao_df.shape)
print('Seville\'s Weather Shape:', w_seville_df.shape)
print('Barcelona\'s Weather Shape:', w_barcelona_df.shape)

Valenci'as Weather Shape: (35064, 17)
Madrid's Weather Shape: (35064, 17)
Bilbao's Weather Shape: (35064, 17)
Seville's Weather Shape: (35064, 17)
Barcelona's Weather Shape: (35064, 17)


In [636]:
# Now lets shrink the energy dataset with the intersection dates
new_energy_df = energy_df[energy_df['time'].isin(intersection_dt)]
new_energy_df = new_energy_df.drop_duplicates(subset=['time'])

print('New Energy Shape:', new_energy_df.shape)

New Energy Shape: (35064, 29)


In [637]:
# List of new weather dataframes that we shrunk
merging_dataframes = [new_energy_df, w_valencia_df, w_barcelona_df, w_bilbao_df, w_madrid_df, w_seville_df]

# Rename all dataframes so that each col has city in feature name
w_valencia_df.columns = [col + "_valencia" if col != 'dt_iso' else col for col in w_valencia_df.columns]
w_barcelona_df.columns = [col + "_barcelona" if col != 'dt_iso' else col for col in w_barcelona_df.columns]
w_seville_df.columns = [col + "_seville" if col != 'dt_iso' else col for col in w_seville_df.columns]
w_bilbao_df.columns = [col + "_bilbao" if col != 'dt_iso' else col for col in w_bilbao_df.columns]
w_madrid_df.columns = [col + "_madrid" if col != 'dt_iso' else col for col in w_madrid_df.columns]

# Merge all datasets with each other
merged_dataset = w_valencia_df.merge(w_barcelona_df, on='dt_iso', how='left')
merged_dataset = merged_dataset.merge(w_bilbao_df, on='dt_iso', how='left')
merged_dataset = merged_dataset.merge(w_madrid_df, on='dt_iso', how='left')
merged_dataset = merged_dataset.merge(w_seville_df, on='dt_iso', how='left')

# Rename energy's time feature to dt_iso
new_energy_df = new_energy_df.rename(columns={'time': 'dt_iso'})

# Merge energy with all 5 weather dataframes
merged_dataset = merged_dataset.merge(new_energy_df, on='dt_iso', how='left')

num_rows, num_columns = merged_dataset.shape
print(num_rows)
print(num_columns)

merged_dataset.head()

35064
109


,dt_iso,city_name_valencia,temp_valencia,temp_min_valencia,temp_max_valencia,pressure_valencia,humidity_valencia,wind_speed_valencia,wind_deg_valencia,rain_1h_valencia,rain_3h_valencia,snow_3h_valencia,clouds_all_valencia,weather_id_valencia,weather_main_valencia,weather_description_valencia,weather_icon_valencia,city_name_barcelona,temp_barcelona,temp_min_barcelona,temp_max_barcelona,pressure_barcelona,humidity_barcelona,wind_speed_barcelona,wind_deg_barcelona,rain_1h_barcelona,rain_3h_barcelona,snow_3h_barcelona,clouds_all_barcelona,weather_id_barcelona,weather_main_barcelona,weather_description_barcelona,weather_icon_barcelona,city_name_bilbao,temp_bilbao,temp_min_bilbao,temp_max_bilbao,pressure_bilbao,humidity_bilbao,wind_speed_bilbao,wind_deg_bilbao,rain_1h_bilbao,rain_3h_bilbao,snow_3h_bilbao,clouds_all_bilbao,weather_id_bilbao,weather_main_bilbao,weather_description_bilbao,weather_icon_bilbao,city_name_madrid,temp_madrid,temp_min_madrid,temp_max_madrid,pressure_madrid,humidity_madrid,wind_speed_madrid,wind_deg_madrid,rain_1h_madrid,rain_3h_madrid,snow_3h_madrid,clouds_all_madrid,weather_id_madrid,weather_main_madrid,weather_description_madrid,weather_icon_madrid,city_name_seville,temp_seville,temp_min_seville,temp_max_seville,pressure_seville,humidity_seville,wind_speed_seville,wind_deg_seville,rain_1h_seville,rain_3h_seville,snow_3h_seville,clouds_all_seville,weather_id_seville,weather_main_seville,weather_description_seville,weather_icon_seville,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation marine,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n,Barcelona,281.625,281.625,281.625,1035,100,7,58,0.0,0.0,0.0,0,800,clear,sky is clear,01n,Bilbao,269.657312,269.657312,269.657312,1036,97,0,226,0.0,0.0,0.0,0,800,clear,sky is clear,01,Madrid,267.325,267.325,267.325,971,63,1,309,0.0,0.0,0.0,0,800,clear,sky is clear,01n,Seville,273.375,273.375,273.375,1039,75,1,21,0.0,0.0,0.0,0,800,clear,sky is clear,01n,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,NaN,863.0,1051.0,1899.0,0.0,7096.0,43.0,73.0,49.0,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n,Barcelona,281.625,281.625,281.625,1035,100,7,58,0.0,0.0,0.0,0,800,clear,sky is clear,01n,Bilbao,269.763500,269.763500,269.763500,1035,97,0,229,0.0,0.0,0.0,0,800,clear,sky is clear,01,Madrid,267.325,267.325,267.325,971,63,1,309,0.0,0.0,0.0,0,800,clear,sky is clear,01n,Seville,273.375,273.375,273.375,1039,75,1,21,0.0,0.0,0.0,0,800,clear,sky is clear,01n,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,NaN,920.0,1009.0,1658.0,0.0,7096.0,43.0,71.0,50.0,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n,Barcelona,281.286,281.286,281.286,1036,100,7,48,0.0,0.0,0.0,0,800,clear,sky is clear,01n,Bilbao,269.251688,269.251688,269.251688,1036,97,1,224,0.0,0.0,0.0,0,800,clear,sky is clear,01,Madrid,266.186,266.186,266.186,971,64,1,273,0.0,0.0,0.0,0,800,clear,sky is clear,01n,Seville,274.086,274.086,274.086,1039,71,3,27,0.0,0.0,0.0,0,800,clear,sky is clear,01n,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,NaN,1164.0,973.0,1371.0,0.0

In [638]:
# Repetitive information since every feature for weather per city has city name in feature name
columns_to_drop = ['city_name_madrid', 'city_name_bilbao', 'city_name_barcelona', 'city_name_seville', 'city_name_valencia', 'temp_min_valencia', 'temp_max_valencia',
                   'temp_min_seville', 'temp_max_seville', 'temp_min_madrid', 'temp_max_madrid', 'temp_min_barcelona', 'temp_max_barcelona', 'temp_min_bilbao', 'temp_max_bilbao']
merged_dataset = merged_dataset.drop(columns=columns_to_drop)
merged_dataset.head()
# merged_dataset.to_csv('merged_energy_weather.csv', index=False)

,dt_iso,temp_valencia,pressure_valencia,humidity_valencia,wind_speed_valencia,wind_deg_valencia,rain_1h_valencia,rain_3h_valencia,snow_3h_valencia,clouds_all_valencia,weather_id_valencia,weather_main_valencia,weather_description_valencia,weather_icon_valencia,temp_barcelona,pressure_barcelona,humidity_barcelona,wind_speed_barcelona,wind_deg_barcelona,rain_1h_barcelona,rain_3h_barcelona,snow_3h_barcelona,clouds_all_barcelona,weather_id_barcelona,weather_main_barcelona,weather_description_barcelona,weather_icon_barcelona,temp_bilbao,pressure_bilbao,humidity_bilbao,wind_speed_bilbao,wind_deg_bilbao,rain_1h_bilbao,rain_3h_bilbao,snow_3h_bilbao,clouds_all_bilbao,weather_id_bilbao,weather_main_bilbao,weather_description_bilbao,weather_icon_bilbao,temp_madrid,pressure_madrid,humidity_madrid,wind_speed_madrid,wind_deg_madrid,rain_1h_madrid,rain_3h_madrid,snow_3h_madrid,clouds_all_madrid,weather_id_madrid,weather_main_madrid,weather_description_madrid,weather_icon_madrid,temp_seville,pressure_seville,humidity_seville,wind_speed_seville,wind_deg_seville,rain_1h_seville,rain_3h_seville,snow_3h_seville,clouds_all_seville,weather_id_seville,weather_main_seville,weather_description_seville,weather_icon_seville,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation marine,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n,281.625,1035,100,7,58,0.0,0.0,0.0,0,800,clear,sky is clear,01n,269.657312,1036,97,0,226,0.0,0.0,0.0,0,800,clear,sky is clear,01,267.325,971,63,1,309,0.0,0.0,0.0,0,800,clear,sky is clear,01n,273.375,1039,75,1,21,0.0,0.0,0.0,0,800,clear,sky is clear,01n,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,NaN,863.0,1051.0,1899.0,0.0,7096.0,43.0,73.0,49.0,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n,281.625,1035,100,7,58,0.0,0.0,0.0,0,800,clear,sky is clear,01n,269.763500,1035,97,0,229,0.0,0.0,0.0,0,800,clear,sky is clear,01,267.325,971,63,1,309,0.0,0.0,0.0,0,800,clear,sky is clear,01n,273.375,1039,75,1,21,0.0,0.0,0.0,0,800,clear,sky is clear,01n,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,NaN,920.0,1009.0,1658.0,0.0,7096.0,43.0,71.0,50.0,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n,281.286,1036,100,7,48,0.0,0.0,0.0,0,800,clear,sky is clear,01n,269.251688,1036,97,1,224,0.0,0.0,0.0,0,800,clear,sky is clear,01,266.186,971,64,1,273,0.0,0.0,0.0,0,800,clear,sky is clear,01n,274.086,1039,71,3,27,0.0,0.0,0.0,0,800,clear,sky is clear,01n,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,NaN,1164.0,973.0,1371.0,0.0,7099.0,43.0,73.0,50.0,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n,281.286,1036,100,7,48,0.0,0.0,0.0,0,800,clear,sky is clear,01n,269.203344,1035,97,1,225,0.0,0.0,0.0,0,800,clear,sky is clear,01,266.186,971,64,1,273,0.0,0.0,0.0,0,800,clear,sky is clear,01n,274.086,1039,71,3,27,0.0,0.0,0.0,0,800,clear,sky is clear,01n,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,NaN,1503.0,949.0,779.0,0.0,7098.0,43.0,75.0,50.0,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,269.686,1002,78,0,23,0.0,0.0,0.0,0

In [639]:
# One hot encoding
merged_dataset = pd.get_dummies(merged_dataset, columns=['weather_main_valencia'])
merged_dataset = pd.get_dummies(merged_dataset, columns=['weather_description_valencia'])
merged_dataset = pd.get_dummies(merged_dataset, columns=['weather_main_seville'])
merged_dataset = pd.get_dummies(merged_dataset, columns=['weather_description_seville'])
merged_dataset = pd.get_dummies(merged_dataset, columns=['weather_main_barcelona'])
merged_dataset = pd.get_dummies(merged_dataset, columns=['weather_description_barcelona'])
merged_dataset = pd.get_dummies(merged_dataset, columns=['weather_main_madrid'])
merged_dataset = pd.get_dummies(merged_dataset, columns=['weather_description_madrid'])
merged_dataset = pd.get_dummies(merged_dataset, columns=['weather_main_bilbao'])
merged_dataset = pd.get_dummies(merged_dataset, columns=['weather_description_bilbao'])

# Not needed information
merged_dataset = merged_dataset.drop(['weather_icon_valencia'], axis=1)
merged_dataset = merged_dataset.drop(['weather_icon_bilbao'], axis=1)
merged_dataset = merged_dataset.drop(['weather_icon_madrid'], axis=1)
merged_dataset = merged_dataset.drop(['weather_icon_seville'], axis=1)
merged_dataset = merged_dataset.drop(['weather_icon_barcelona'], axis=1)

In [640]:
# Calculate the correlation matrix
corr_matrix = merged_dataset.drop(columns='dt_iso').corr()

# Set a correlation threshold for considering values as highly correlated
correlation_threshold = 0.9 

# Find highly correlated pairs
highly_correlated_pairs = []

# Iterate through the correlation matrix
for i in range(len(corr_matrix.columns)):
    for j in range(i + 1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > correlation_threshold:
            pair = (corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j])
            highly_correlated_pairs.append(pair)

# Print highly correlated pairs
print("Highly Correlated Pairs:")
for pair in highly_correlated_pairs:
    print(pair)

# Dropping all features with a 1.0 correlation to weather_main_city
merged_dataset = merged_dataset.drop(['weather_description_valencia_sky is clear'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_valencia_light intensity drizzle'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_valencia_fog'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_valencia_haze'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_valencia_mist'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_valencia_smoke'], axis=1)

merged_dataset = merged_dataset.drop(['weather_description_seville_sky is clear'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_seville_light intensity drizzle'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_seville_fog'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_seville_haze'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_seville_mist'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_seville_smoke'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_seville_dust'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_seville_squalls'], axis=1)

merged_dataset = merged_dataset.drop(['weather_description_madrid_sky is clear'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_madrid_light intensity drizzle'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_madrid_fog'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_madrid_haze'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_madrid_mist'], axis=1)

merged_dataset = merged_dataset.drop(['weather_description_barcelona_sky is clear'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_barcelona_light intensity drizzle'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_barcelona_fog'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_barcelona_mist'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_barcelona_sand dust whirls'], axis=1)

merged_dataset = merged_dataset.drop(['weather_description_bilbao_sky is clear'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_bilbao_light intensity drizzle'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_bilbao_fog'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_bilbao_haze'], axis=1)
merged_dataset = merged_dataset.drop(['weather_description_bilbao_mist'], axis=1)

# Dropping forecasted features, have a ~0.995 correlation to the "actual" feature
merged_dataset = merged_dataset.drop(['forecast solar day ahead'], axis=1) # generation solar
merged_dataset = merged_dataset.drop(['forecast wind onshore day ahead'], axis=1) # generation wind onshore
merged_dataset = merged_dataset.drop(['total load forecast'], axis=1) # total load actual


Highly Correlated Pairs:
('temp_valencia', 'temp_barcelona', 0.9175055701292097)
('temp_valencia', 'temp_madrid', 0.912280889603956)
('temp_barcelona', 'temp_madrid', 0.9039959494830988)
('temp_madrid', 'temp_seville', 0.9094213472651101)
('generation solar', 'forecast solar day ahead', 0.9932644415637772)
('generation wind onshore', 'forecast wind onshore day ahead', 0.9944591935562195)
('total load forecast', 'total load actual', 0.9951389210060558)
('weather_main_valencia_clear', 'weather_description_valencia_sky is clear', 1.0)
('weather_main_valencia_drizzle', 'weather_description_valencia_light intensity drizzle', 0.9339116551720714)
('weather_main_valencia_fog', 'weather_description_valencia_fog', 1.0)
('weather_main_valencia_haze', 'weather_description_valencia_haze', 1.0)
('weather_main_valencia_mist', 'weather_description_valencia_mist', 1.0)
('weather_main_valencia_smoke', 'weather_description_valencia_smoke', 1.0)
('weather_main_seville_clear', 'weather_description_seville_

In [641]:
# Rerun code: now these are our most highly correlated features, but let's keep them
corr_matrix = merged_dataset.drop(columns='dt_iso').corr()

correlation_threshold = 0.9

# Find highly correlated pairs
highly_correlated_pairs = []

# Iterate through the correlation matrix
for i in range(len(corr_matrix.columns)):
    for j in range(i + 1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > correlation_threshold:
            pair = (corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j])
            highly_correlated_pairs.append(pair)

# Print highly correlated pairs that we acknowledge are highly correlated but we want to keep
# because city and temperature are important to us
print("Highly Correlated Pairs:")
for pair in highly_correlated_pairs:
    print(pair)

Highly Correlated Pairs:
('temp_valencia', 'temp_barcelona', 0.9175055701292097)
('temp_valencia', 'temp_madrid', 0.912280889603956)
('temp_barcelona', 'temp_madrid', 0.9039959494830988)
('temp_madrid', 'temp_seville', 0.9094213472651101)


In [642]:
missing_values = merged_dataset.isnull()

# Count the number of missing values per column
num_missing_per_column = missing_values.sum()

# Filter and print only the columns with missing values
missing_columns = num_missing_per_column[num_missing_per_column > 0]
print("Columns with Missing Values:")
print(missing_columns)

Columns with Missing Values:
generation biomass                                19
generation fossil brown coal/lignite              18
generation fossil coal-derived gas                18
generation fossil gas                             18
generation fossil hard coal                       18
generation fossil oil                             19
generation fossil oil shale                       18
generation fossil peat                            18
generation geothermal                             18
generation hydro pumped storage aggregated     35064
generation hydro pumped storage consumption       19
generation hydro run-of-river and poundage        19
generation hydro water reservoir                  18
generation marine                                 19
generation nuclear                                17
generation other                                  18
generation other renewable                        18
generation solar                                  18
generation waste 

In [643]:
missing_indices = merged_dataset[merged_dataset['generation biomass'].isna()].index

# Print the record IDs (indices) with missing values 
print(f"Record IDs with missing values for {'generation biomass'}:")
print(missing_indices)

missing_indices = merged_dataset[merged_dataset['generation fossil brown coal/lignite'].isna()].index

# Print the record IDs (indices) with missing values 
print(f"Record IDs with missing values for {'generation fossil brown coal/lignite'}:")
print(missing_indices)

missing_indices = merged_dataset[merged_dataset['generation marine'].isna()].index

# Print the record IDs (indices) with missing values
print(f"Record IDs with missing values for {'generation marine'}:")
print(missing_indices)

# There are several records that are missing a contiguous set of features, but we will impute these anyways

Record IDs with missing values for generation biomass:
Index([   99,   108,   109,   110,   111,   112,   113,   451,   452,   643,
         661,  2528,  2708,  3968,  6586,  8049, 13341, 16612, 30896],
      dtype='int64')
Record IDs with missing values for generation fossil brown coal/lignite:
Index([   99,   108,   109,   110,   111,   112,   113,   451,   452,   643,
         661,  2528,  2708,  3968,  6586,  8049, 13341, 30896],
      dtype='int64')
Record IDs with missing values for generation marine:
Index([   99,   108,   109,   110,   111,   112,   113,   451,   452,   643,
         661,  2528,  2708,  3968,  6586,  8049, 13341, 15598, 30896],
      dtype='int64')


In [644]:
# Convert "dt_iso" to a recognized datetime format
# changes time zone from CET to UTC (+01:00 to +:00:00)
# same time though just time zone changed for grouping by date
merged_dataset['dt_iso'] = pd.to_datetime(merged_dataset['dt_iso'], utc=True)

# Calculate the daily average of "generation biomass" for each date
daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation biomass'].transform('mean')
# Fill missing values with the daily average
merged_dataset['generation biomass'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation fossil brown coal/lignite'].transform('mean')
merged_dataset['generation fossil brown coal/lignite'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation fossil coal-derived gas'].transform('mean')
merged_dataset['generation fossil coal-derived gas'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation fossil gas'].transform('mean')
merged_dataset['generation fossil gas'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation fossil hard coal'].transform('mean')
merged_dataset['generation fossil hard coal'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation fossil oil'].transform('mean')
merged_dataset['generation fossil oil'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation fossil oil shale'].transform('mean')
merged_dataset['generation fossil oil shale'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation fossil peat'].transform('mean')
merged_dataset['generation fossil peat'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation geothermal'].transform('mean')
merged_dataset['generation geothermal'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation hydro pumped storage consumption'].transform('mean')
merged_dataset['generation hydro pumped storage consumption'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation hydro run-of-river and poundage'].transform('mean')
merged_dataset['generation hydro run-of-river and poundage'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation hydro water reservoir'].transform('mean')
merged_dataset['generation hydro water reservoir'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation marine'].transform('mean')
merged_dataset['generation marine'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation nuclear'].transform('mean')
merged_dataset['generation nuclear'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation other'].transform('mean')
merged_dataset['generation other'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation other renewable'].transform('mean')
merged_dataset['generation other renewable'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation waste'].transform('mean')
merged_dataset['generation waste'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation wind offshore'].transform('mean')
merged_dataset['generation wind offshore'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation wind onshore'].transform('mean')
merged_dataset['generation wind onshore'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['forecast wind offshore eday ahead'].transform('mean')
merged_dataset['forecast wind offshore eday ahead'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['total load actual'].transform('mean')
merged_dataset['total load actual'].fillna(daily_averages, inplace=True)

daily_averages = merged_dataset.groupby(merged_dataset['dt_iso'].dt.date)['generation solar'].transform('mean')
merged_dataset['generation solar'].fillna(daily_averages, inplace=True)

In [645]:
missing_values = merged_dataset.isnull()

# Count the number of missing values per column
num_missing_per_column = missing_values.sum()

# Filter and print only the columns with missing values
missing_columns = num_missing_per_column[num_missing_per_column > 0]
print("Columns with Missing Values:")
print(missing_columns)

# Now only two columns can be used for feature engineering if we so wish, which are completely empty

Columns with Missing Values:
generation hydro pumped storage aggregated    35064
forecast wind offshore eday ahead             35064
dtype: int64
